In [135]:
import requests
import json
import re
from bs4 import BeautifulSoup
import urlextract

# 使用方式：
# 先鎖定要開始爬取的文章，比方說從2022年1月1號開始往2023年爬，那就找出2022年的第一篇文章所在的文章列表的頁面網址，以及該篇文章的標題。
# 1.修改調用Set_Data時的傳入網址
# 2.修改傳入文章標題


# 注意！！！IndexError被我例外化了，由於ptt的文章可以自由編輯，整個文章架構都能被亂刪除，像是推文跟內文直接被砍掉，只剩下殘缺的格式，容易造成index error，所以我將index error的部分直接跳過了。
# 若編寫程式有需要index操作，建議先將Set_Data最後方的except那邊註解掉，方便debug。
# except IndexError as e: 
#     print("error:",e)
#     continue



#去除超連結，實測推文和文章中還經常有：
# https://reurl
# https://imgur.com
# https://i.imgur.com
# https://www.ptt.cc
# 等網址沒消除，有需要的話自己再除錯看看，反正問gpt就是了。
def remove_hyperlinks(text):
    extractor = urlextract.URLExtract()
    urls = extractor.find_urls(text)
    for url in urls:
        text = text.replace(url, '')
    return text


DataList=[]


#tset url:https://www.ptt.cc/bbs/DailyArticle/M.1577930501.A.C92.html
#2020第一篇文章網址
#
# https://www.ptt.cc/bbs/DailyArticle/index594.html
# 該文章所在頁面
#
# [
#     {
#         "Title": "有人吃過嚴萃葉黃素嗎?",
#         "Type": "health",
#         "Author": "Kitty3313 (嗨)",
#         "Date": "Thu Jul 20 13:13:30 2023",
#         "Body": "爬文看了板上分享葉黃素要選游離型分子較小...(以下省略)",
#         "comment": ["太神啦", "原神啟動！"],
#         "Url": "https://www.ptt.cc/bbs/regimen/M.1689830012.A.862.html"
#     }
# ]

def Set_Data(url,first_title):
    Number=int(re.search('index(\d+)\.html',url).group(1)) #用來迭代頁數的變數，抓取輸入網址中的頁數
    
    startup=False
    while (True):
        try:
            print(re.sub('index\d+\.html', f'index{Number}.html', url))
            r=requests.get(re.sub('index\d+\.html', f'index{Number}.html', url)) #用正則表達式處理網址
            Number+=1
            
            if r.status_code == 404: #停止爬蟲條件
                print("404 錯誤：頁面不存在。")
                break
                
            soup=BeautifulSoup(r.text,"html.parser") #載入文章列表網址
            sel=soup.select(".r-ent")
            
            for i in sel:
                Json_Dict={}
                Json_Dict["Type"]="life"
                

                pattern=r"](.*?)\n"
                Json_Dict["Title"]=re.search(pattern,i.select_one("div.title").text, re.DOTALL).group(1).strip()
                
                if(Json_Dict["Title"]!=first_title and not startup): #你要抓的第一篇通常並不在該頁的第一篇，故省略不需要的文章直到第一篇開始。
                    continue
                else: startup=True
                    
                if(not Json_Dict["Title"]):continue #看是否有取到標題，問題通常發生在文章被版主刪除而系統尚未清除時((本文已被刪除) [getniceone])。
                print(Json_Dict["Title"])
                    
                Json_Dict["Url"]="https://www.ptt.cc"+i.select_one("div.title a")['href']
                print(Json_Dict["Url"])
                
                Json_Dict["Author"]=i.select_one("div.meta div.author").text

                article=requests.get(Json_Dict["Url"]) #載入文章網址
                article_soup=BeautifulSoup(article.text,"html.parser")

                Json_Dict["Date"]=article_soup.select("div.article-metaline")[2].text[2:] #這邊用切片切掉了前綴時間兩字

                article_text=article_soup.select("div#main-content")[0].text
                pattern = r"\d{2}:\d{2}:\d{2}\s\d{4}\n(.*?)--\n※ 發信站:" #ptt的內文沒有分隔，只能用正則把他取出來
                Json_Dict["Body"]= re.search(pattern, article_text, re.DOTALL).group(1).strip() #取出後用group(1)匹配第一個()抓到的內容，在用strip()去除頭尾的空白、換行等符號
                Json_Dict["Body"]=remove_hyperlinks(Json_Dict["Body"]) #去除超連結
                Json_Dict["Body"]=re.sub(r'[^\u4e00-\u9fff，。、；：「」『』（）《》？！\s]', '', Json_Dict["Body"]) #去除特殊符號
                Json_Dict["Body"]=re.sub(r'\n+', '\n',Json_Dict["Body"]) #多個換行符號轉成單個
                Json_Dict["Body"]=re.sub('\\s+', ' ',Json_Dict["Body"]) #多個空白符號轉成單個
                
                pattern=":\s(.*?)\s\d{2}/\d{2}"
                if(re.findall(pattern, article_text)): #判斷是否有推文
                    Json_Dict["comment"]=re.findall(pattern, article_text) #抓取推文
                    for i in Json_Dict["comment"]:
                        i=remove_hyperlinks(i) #去除超連結
                        i=re.sub(r'[^\u4e00-\u9fff，。、；：「」『』（）《》？！\s]', '', i) #去除特殊符號
                        i=re.sub(r'\n+', '\n',i) #多個換行符號轉成單個
                        i=re.sub('\\s+', ' ',i) #多個空白符號轉成單個
                else: Json_Dict["comment"]="" #沒推文就給個空字元吧
                    
                print(Json_Dict)
                DataList.append(Json_Dict)
                
            print("已爬取文章數:"+str(len(DataList)))

        except AttributeError as e: 
            print("error:",e)
            continue #由於用到的 group(1)函數，當我們正則沒取到值時會跳出此錯誤，直接捨棄文章不做處理。
            
        except IndexError as e: # 若編寫程式有需要index操作，建議先將Set_Data最後方的except那邊註解掉，方便debug。
            print("error:",e)
            continue

Set_Data("https://www.ptt.cc/bbs/Tainan/index5013.html","水桶") #填入要開始爬取的頁面和起始文章標題。
    



https://www.ptt.cc/bbs/Tainan/index5013.html
水桶
https://www.ptt.cc/bbs/Tainan/M.1691132248.A.357.html
{'Type': 'life', 'Title': '水桶', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691132248.A.357.html', 'Author': 'Bignana', 'Date': 'Fri Aug  4 14:57:25 2023', 'Body': ' 暫停 發言，期限為 天 理由 水桶 暫停 發言，期限為 天 理由 水桶 渡五趣、定四正、歸三悟，薩埵十二惡皆空。', 'comment': ['[水桶]xkirua 1-1-2']}
台南18~30歲股民面訪 車馬費1600元
https://www.ptt.cc/bbs/Tainan/M.1691132559.A.04D.html
{'Type': 'life', 'Title': '台南18~30歲股民面訪 車馬費1600元', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691132559.A.04D.html', 'Author': 'Tror1', 'Date': 'Fri Aug  4 15:02:37 2023', 'Body': '地點台南市全區 徵求內容 台南台股交易研究一對一面訪 車馬費 小時 招募對象條件： 目前居住在台南，歲男女 本人有任何一家劵商帳號，且投資操作也是本人 今年年 月有台股交易買賣經驗 可配合提供台股「過去半年歷史交易紀錄」及「目前庫存股」截圖 報名連結： 訪談時間： 一 三 訪問地點另行與受訪者約定方便面訪處 車馬費： 小時 訪談結束後，隔日資料填寫完整後匯款 注意事項 網路報名成功不代表合格，若初步不合格不會再電話通知。 所有活動均不接受電話報名，欲報名請依網頁指示報名！ 經過專案人員初步的審核後，將會有專員電話與您進行電話審核，請注意市調公司電話致 電訪問 電話訪問合格者經主管篩選後，會再次以電話邀請！ 傳令市場顧問有限公司 以下為加強提醒請勿刪除發文前請詳閱並遵守板規 私人 無涉及金錢任何事務 徵求 不可重 台南限定 謝禮有限額 面交地點請填區名或明顯地標，只填

{'Type': 'life', 'Title': '請問現在酒吧室內全面禁菸嗎？', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691144211.A.4E2.html', 'Author': 'c753968412', 'Date': 'Fri Aug  4 18:16:49 2023', 'Body': '如題 本身高雄人 想去台南亂一下吧 想說收尾找間能抽菸的帶雪茄去 但翻了之前的文章找能室內抽煙的 他們回覆都是室內禁菸 想詢問板上有沒有推薦目前還能在室內吸煙且調酒味道不錯的吧呢？ （小聲說怎麼高雄一些店家還是可以在室內用水煙跟吸菸啊，是有關係就沒關係嗎？', 'comment': ['c753968412 (36.238.168.194 臺灣),', 'c753968412 (36.238.168.194 臺灣),', '抽菸bad', '上個月我去高雄lamp 裡面可以抽菸還有在賣 然後我中途', '下樓要再上去的時候叼著菸被顧門口擋住說裡面禁煙 給', '你參考一下', 'One bar', '有些事可以做 不可以說  有些事可以說不可以做', '最新的修法酒吧夜店都禁煙 店家會覺得你來釣魚的吧', '台灣室內不是全面禁菸了嗎', '幫警察蒐集臨檢名單的概念', '不要說 會被偷偷查 抓到就是開罰一萬']}
衝中秋節買氣！台南10家身障團體獲頒「台南好心意」標章
https://www.ptt.cc/bbs/Tainan/M.1691144244.A.AB3.html
{'Type': 'life', 'Title': '衝中秋節買氣！台南10家身障團體獲頒「台南好心意」標章', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691144244.A.AB3.html', 'Author': 'jackloutter', 'Date': 'Fri Aug  4 18:17:22 2023', 'Body': '衝中秋節買氣！台南家身障團體獲頒「台南好心意」標章 聯合報 記者 吳淑玲 台南即時報導 台南市長黃偉哲（右三）頒發「台南好心意」標章給家身心障礙機構及團體，邀請民眾 踴躍選購。記者吳淑玲攝影 影片 中秋節即將來臨，為增加身障機構團體秋節禮盒買氣，台南市政府社會局舉辦「台南好

{'Type': 'life', 'Title': '台南火車站內部多處正在漏水', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691147455.A.6C2.html', 'Author': 'teramars', 'Date': 'Fri Aug  4 19:10:53 2023', 'Body': '剛剛跟朋友去台南火車站準備搭車的時候 發現在地上有放了一些水桶 抬頭一看 原來是在漏水 但是現在外面也沒下雨 地面完全沒有下雨的痕跡 不知道為什麼還是一直在漏水？ ', 'comment': ['怎麼越來越多桶了 笑死 上週看只有一桶', '水管漏水了吧', '之前下雨吸飽水份開始吐水', '這個車站已經沒救了......', '古蹟', 'XDDD 爛透了 裡面水桶好像一直都在 只是多寡而已', '綠色水桶充滿台灣價值', '這是什麼工地', '台南火車站有二樓啊，之前修復古蹟被敲掉後就停工到現在', '，二樓一般民眾根本上不去，會漏水估計是有積水', '這個也要抓內鬼嗎', '這車站到底修多久了', '就被修壞了啊，然後跟包商還在打官司不是？', '這就是死忠的一直再吹的古都', '笑死 一個車站爛成這樣', '從20多年前常搭火車時 下大雨必見場景', '確實修很久了，只能用慘字形容', '扯', '打掉重建卡實在', '又要開始抓內鬼了是嗎？', '這算台鐵的鍋 還是西瓜沒魄力? 六都車站輸斗六車站', '觀光客最多的台南 結果車站破成這樣', '這台鐵的工程，要罵西瓜要去歸仁文化中心那篇。', '內鬼？藍白拖？中國共鳴者？', '搞成這樣不如不修 有夠誇張', '這個有積水的話，不去噴藥嗎？', '台鐵的問題台南幾個民進黨立委應該要出來盯，台南', '建城四百年在即，不論工程狀況如何，應該先拆除鷹架', '復原，而不是讓台南門面之一一直爛在那邊', '說個笑話，古都把古蹟修壞了', '登革熱防治落實四步驟巡、倒、清、刷。', '哭啊 什麼時候會好', '忘記去年還前年了，正門入口右邊拱門的天花板因為下', '雨潮濕，部分脫落掉下來，懷疑工程真的有做防水？', '室內沒下雨還漏水，大概率是空調出問題啦', '古蹟日常', '日常', '噴藥笑死', '這是有台灣價值的漏水啦 你懂什麼？！', '唉 基隆車站說拆就拆 

{'Type': 'life', 'Title': '2006年凱米颱風為何臺南不放假', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691164494.A.92B.html', 'Author': 'Kbart', 'Date': 'Fri Aug  4 23:54:49 2023', 'Body': '當年還是小高中生 對這件事至今還是耿耿於懷，雖然賴市長時代這邊的爭議更多但那時候我並不在臺南 反而印象不深 故事回到晚上 凱米颱風預估會直接穿過臺灣本島 由於北部影響不大，因此北部是宣布照常上班上課 颱風中心從臺東登陸，一路貫穿至嘉義出海 因此嘉義周邊的臺南高雄理論上是風雨影響最大的範圍 可是這是隔天的宣布結果 整個南臺灣，僅風雨影響最大的臺南縣市居然是宣布照常上班上課？ 為何凱米颱風臺南縣市仍宣布照常上班上課？ 回顧颱風路徑，颱風風暴最強的區域可是直接穿過臺南耶！ 我還沒說完吶 夠了夠了，回家啦 ', 'comment': ['也太久 紅杉軍年代的事情了', '感覺某成大自強的系館玻璃門被風吹倒那年？沒什麼印像了', 'ㄌㄨㄚˋ', '這路線反而通常不大會有雨', '印像以前很少因為風的因素停班停課 都是看雨居多', '就我所知地方政府防災中心的啟動機制 一般是颱風警戒跟暴', '雨警戒 感覺今年風力影響被提升到放假決策影響力', '我是覺得今年選舉年大家都在討民眾開心 不然 西瓜跟萬安最', '近 做的跟賴半天才放假那次有什麼不同 但輿論完全不同', '更正 誤會萬安 我指西瓜第二天放假那次 也是賴了半天', '幫幫忙，都幾年了！', '哪有 也是一樣6點正式發佈', '一個颱風你記了17年，怨念也太深', 'Kbart (1.175.166.178 臺灣),', '以前都是早上才在看有沒有放假的 我對賴不放', '比較有怨言 我記得第一次他不放被塞到', '我們有自己的成大氣象團隊 然後就', '賴神勸童了 話真的不要說太滿 這次第二天又放滿出乎我意', '外的', '我印象中從我高中大學出社會在我後期幾乎台南都沒在放', '颱風假的', '覺得518那次誤放颱風假可能是關鍵，就我自己的印象在這之', '後臺南的爭議幾乎都是狂風暴雨不放假，鮮少聽到無風無雨卻', '放颱風假這種爭議', '穿心颱風雨都被中央山脈擋住了吧

{'Type': 'life', 'Title': '售 osim智能腿腿樂2', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691169421.A.DC4.html', 'Author': 'goldapple', 'Date': 'Sat Aug  5 01:16:59 2023', 'Body': '交易類別出售 面交地點須為台南市內 永康區、北區 交易物品金額說明 智能 腿腿樂出售，保存狀況約成新 今年月初在新天地購入， 都沒在用故割愛售出 仍在保固期內，售元 意者請站內信 ', 'comment': ''}
搬家出清 床頭櫃
https://www.ptt.cc/bbs/Tainan/M.1691170859.A.F12.html
{'Type': 'life', 'Title': '搬家出清 床頭櫃', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.1691170859.A.F12.html', 'Author': 'j22953111', 'Date': 'Sat Aug  5 01:40:57 2023', 'Body': '交易類別出售 面交地點須為台南市內安南區近北區 交易物品金額說明 床頭櫃 售元 表面有使用痕跡 上層內部不知道卡到什麼東西 只能打開一半，但不影響收納 下層正常 因材質木頭有些笨重，需開車載 以下為加強提醒請勿刪除發文前請詳閱並遵守板規 私人 涉及金錢任何事務 交易 天限一篇 不可重覆 限定台南 面交地點請填區名或明顯地標，只填台南表示台南市內個區皆可面交唷 交易完成可在標題或內文附註，但勿更改交易主標題 、票券、卡片與收集類請推置底文，違規者會被水桶，請勿以身試法 、等網路類會員算在類不限於以上兩者，違規會被水桶 請注意台南板主要為分享討論性質看板，開放二手物交易為附屬，請勿濫用交易。 ', 'comment': ''}
08/05(六)~08/06(日) 捐血活動
https://www.ptt.cc/bbs/Tainan/M.1691174638.A.2C7.html
{'Type': 'life', 'Title': '08/05(六)~08/06(日) 捐血活動', 'Url': 'https://www.ptt.cc/bbs/Tainan/M.

In [125]:
with open("output"+".json","w",encoding='utf-8')as f:
    json.dump(DataList,f,indent=4,ensure_ascii=False) #存檔
print("---結束---")

---結束---
